In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
import pandas as pd
%matplotlib inline

In [16]:
cities = pd.read_csv("cities_data.csv", index_col=False)
if 'Unnamed: 0' in cities.columns:
    cities = cities.drop('Unnamed: 0', axis=1)

cities = cities.dropna()

In [ ]:
city_list = cities['city'].to_list()
continent_list = cities['REGION'].to_list()

48057

In [ ]:
#decode/encode cities
chars = sorted(list(set(''.join(city_list))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

def decode_city(word:str):
    decoded = [stoi[i] for i in word]
    return decoded

def encode_city(word:list[int]):
    encoded = ''.join(itos[i] for i in word)
    return encoded

In [41]:
decode_city('ho chi minh city')
#encode_city([30, 25, 21, 35, 25])

[18, 25, 1, 13, 18, 19, 1, 23, 19, 24, 18, 1, 13, 19, 30, 35]

In [86]:
#decode/encode continents
stoi_1 = {s: i+(len(chars)) for i, s in enumerate(set(continent_list))}
stoi_1
itos_1 = {i:s for s, i in stoi_1.items()}
itos_1

def decode_continent(word:str):
    decoded = stoi_1[word]
    return decoded

def encode_continent(number):
    encoded = itos_1[number]
    return encoded

stoi_1

{'ASIA': 164,
 'ANTARCTICA': 165,
 'OCEANIA': 166,
 'AFRICA': 167,
 'SOUTH AMERICA': 168,
 'NORTH AMERICA': 169,
 'EUROPE': 170}

In [ ]:
#Build Dataset

from typing import Literal

def build_dataset(continent_list, city_list):
  block_size = 3 # context length: how many characters do we take to predict the next one?
  X, Y = [], []
  for index, city in enumerate(city_list):
    
    #print(w)
    ic = [decode_continent(continent_list[index])]
    context = ic + [0] * block_size 
    for ch in city + '.':
      ix = decode_city(ch)
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = [context[0]] + context[2:] + ix #crop and append
      
  X = torch.tensor(X)
  Y = torch.tensor(Y)

  return X, Y


def split_data(continent_list, city_list):
  import random
  random.seed(42)
  random.shuffle(city_list)
  n1 = int(0.8*len(city_list))
  n2 = int(0.9*len(city_list))

  Xtr, Ytr = build_dataset(continent_list[:n1], city_list[:n1])
  Xdev, Ydev = build_dataset(continent_list[n1:n2], city_list[n1:n2])
  Xte, Yte = build_dataset(continent_list[n2:], city_list[n2:])


In [90]:
import torch
import torch.nn.functional as F

In [ ]:
#training begins #MLP

import torch
from torch import nn
import torch.nn.functional as F


n_embed = 10 # dimension of the vector space for embedding
n_hidden = 200 # number of neurons in the hidden layer
vocab_size = len(chars) + len(set(continent_list))
block_size = 4
batch_size = 50


class FirstMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        self.mlp = nn.Sequential(
            nn.Embedding(num_embeddings = vocab_size, embedding_dim = n_embed), #(50, 4, 10)
            nn.Flatten(start_dim = 1), #(50, 40)

            nn.BatchNorm1d(),

        )

    # def forward(self, x):
    #     x = self.flatten(x)
    #     logits = self.linear_relu_stack(x)
    #     return logits

NameError: name 'nn' is not defined

171